In [1]:
import psycopg2
import pandas as pd

username = "postgres"
password = "test"
hostname = "db"
port = "5432"
database = "test"

dst = f"postgresql://{username}:{password}@{hostname}:{port}/{database}"
con = psycopg2.connect(dst)

In [11]:
# create table
def create_shohin_table(cur):
    table_name = 'shohin'
    cur.execute(f"drop table if exists {table_name}")
    cur.execute(f"""create table {table_name} (
        shohin_id char(4) not null,
        shohin_mei varchar(100) not null,
        shohin_bunrui varchar(32) not null,
        hanbai_tanka integer,
        shiire_tanka integer,
        torokubi date,
        primary key (shohin_id));""")
    return cur

with psycopg2.connect(dst) as con, con.cursor() as cur:
    try:
        create_shohin_table(cur)
        sql = f"select * from shohin"
        print(pd.read_sql(sql, con=con))
    except psycopg2.Error as er:
        print('psycopg2.Error:', er)

Empty DataFrame
Columns: [shohin_id, shohin_mei, shohin_bunrui, hanbai_tanka, shiire_tanka, torokubi]
Index: []


In [19]:
# drop table
with psycopg2.connect(dst) as con, con.cursor() as cur:
    table_name = 'shohin'
    try:
        cur.execute(f"drop table {table_name};")
        sql = f"select * from shohin"
        print(pd.read_sql(sql, con=con))
    except pd.io.sql.DatabaseError as e:
        print(f"expected error {e}") 

expected error Execution failed on sql 'select * from shohin': relation "shohin" does not exist
LINE 1: select * from shohin
                      ^



In [24]:
# arrange column
with psycopg2.connect(dst) as con, con.cursor() as cur:
    table_name = 'shohin'
    try:
        create_shohin_table(cur)
        
        sql = f"select * from {table_name};"
        print(pd.read_sql(sql, con=con))
        
        cur.execute(f"alter table {table_name} add column shohin_mai_kana varchar(100)")
        print(pd.read_sql(sql, con=con))
        
        cur.execute(f"alter table {table_name} drop column shohin_mai_kana;")
        print(pd.read_sql(sql, con=con))
    except psycopg2.Error as e:
        print(f"unexpected error: {e}")

Empty DataFrame
Columns: [shohin_id, shohin_mei, shohin_bunrui, hanbai_tanka, shiire_tanka, torokubi]
Index: []
Empty DataFrame
Columns: [shohin_id, shohin_mei, shohin_bunrui, hanbai_tanka, shiire_tanka, torokubi, shohin_mai_kana]
Index: []
Empty DataFrame
Columns: [shohin_id, shohin_mei, shohin_bunrui, hanbai_tanka, shiire_tanka, torokubi]
Index: []


In [40]:
# register data list
with psycopg2.connect(dst) as con, con.cursor() as cur:
    table_name = 'shohin'
    try:
        create_shohin_table(cur)
        
        table_sql = f"select * from {table_name};"
        print(pd.read_sql(table_sql, con=con))
        
        insert_sql = f"""begin transaction;
            insert into {table_name} values ('0001', 'Tシャツ',       '衣服',        1000, 500,   '2009-09-20');
            insert into {table_name} values ('0002', '穴あけパンチ',   '事務用品',     500,  320,   '2009-09-11');
            insert into {table_name} values ('0003', 'カッターシャツ',  '衣服',       4000,  2800,  null);
            insert into {table_name} values ('0004', '包丁',          'キッチン用品', 3000, 2800,  '2009-09-20');
            insert into {table_name} values ('0005', '圧力鍋',        'キッチン用品', 6800,  5000, '2009-01-15');
            insert into {table_name} values ('0006', 'フォーク',       'キッチン用品', 500,  null, '2009-09-20');
            insert into {table_name} values ('0007', 'おろしがね',     'キッチン用品', 880,  790,  '2008-04-28');
            insert into {table_name} values ('0008', 'ボールペン',     '事務用品',    100,   null, '2009-11-11');
            commit;"""
        cur.execute(insert_sql)
        print(pd.read_sql(table_sql, con=con))
    except psycopg2.Error as e:
        print(f"unexpected error: {e}")

Empty DataFrame
Columns: [shohin_id, shohin_mei, shohin_bunrui, hanbai_tanka, shiire_tanka, torokubi]
Index: []
  shohin_id shohin_mei shohin_bunrui  hanbai_tanka  shiire_tanka torokubi   
0  0001         Tシャツ        衣服        1000           500.0        2009-09-20
1  0002       穴あけパンチ      事務用品         500           320.0        2009-09-11
2  0003      カッターシャツ        衣服        4000          2800.0              None
3  0004           包丁    キッチン用品        3000          2800.0        2009-09-20
4  0005          圧力鍋    キッチン用品        6800          5000.0        2009-01-15
5  0006         フォーク    キッチン用品         500             NaN        2009-09-20
6  0007        おろしがね    キッチン用品         880           790.0        2008-04-28
7  0008        ボールペン      事務用品         100             NaN        2009-11-11


In [41]:
# select column
pd.options.display.colheader_justify = 'right'
with psycopg2.connect(dst) as con, con.cursor() as cur:
    table_name = 'shohin'
    try:
        sql = f"select shohin_mei, shiire_tanka from {table_name};"
        print(pd.read_sql(sql, con=con))
    except psycopg2.Error as e:
        print(f"unexpected error: {e}")

  shohin_mei  shiire_tanka
0       Tシャツ         500.0
1     穴あけパンチ         320.0
2    カッターシャツ        2800.0
3         包丁        2800.0
4        圧力鍋        5000.0
5       フォーク           NaN
6      おろしがね         790.0
7      ボールペン           NaN


In [43]:
# select column as original names
with psycopg2.connect(dst) as con, con.cursor() as cur:
    table_name = 'shohin'
    try:
        sql = f"""select shohin_id    as id,
                         shohin_mei   as namae,
                         shiire_tanka as tanka
                    from shohin"""
        print(pd.read_sql(sql, con=con))
    except psycopg2.Error as e:
        print(f"unexpected error: {e}")

     id    namae   tanka
0  0001     Tシャツ   500.0
1  0002   穴あけパンチ   320.0
2  0003  カッターシャツ  2800.0
3  0004       包丁  2800.0
4  0005      圧力鍋  5000.0
5  0006     フォーク     NaN
6  0007    おろしがね   790.0
7  0008    ボールペン     NaN


In [44]:
# output constants
with psycopg2.connect(dst) as con, con.cursor() as cur:
    table_name = 'shohin'
    try:
        sql = f"""select '商品'       as mohiretsu,
                         38           as kazu,
                         shohin_id    as id,
                         shohin_mei   as namae,
                         shiire_tanka as tanka
                    from shohin"""
        print(pd.read_sql(sql, con=con))
    except psycopg2.Error as e:
        print(f"unexpected error: {e}")

  mohiretsu  kazu    id    namae   tanka
0        商品    38  0001     Tシャツ   500.0
1        商品    38  0002   穴あけパンチ   320.0
2        商品    38  0003  カッターシャツ  2800.0
3        商品    38  0004       包丁  2800.0
4        商品    38  0005      圧力鍋  5000.0
5        商品    38  0006     フォーク     NaN
6        商品    38  0007    おろしがね   790.0
7        商品    38  0008    ボールペン     NaN


In [46]:
# distinction
with psycopg2.connect(dst) as con, con.cursor() as cur:
    table_name = 'shohin'
    try:
        sql = f"""select shohin_bunrui
                    from shohin"""
        print(pd.read_sql(sql, con=con))
        
        sql = f"""select distinct shohin_bunrui
                    from shohin"""
        print(pd.read_sql(sql, con=con))
    except psycopg2.Error as e:
        print(f"unexpected error: {e}")

  shohin_bunrui
0            衣服
1          事務用品
2            衣服
3        キッチン用品
4        キッチン用品
5        キッチン用品
6        キッチン用品
7          事務用品
  shohin_bunrui
0        キッチン用品
1            衣服
2          事務用品


In [47]:
# distinction with null data
with psycopg2.connect(dst) as con, con.cursor() as cur:
    table_name = 'shohin'
    try:
        sql = f"""select shiire_tanka
                    from {table_name}"""
        print(pd.read_sql(sql, con=con))
        
        sql = f"""select distinct shiire_tanka
                    from {table_name}"""
        print(pd.read_sql(sql, con=con))
    except psycopg2.Error as e:
        print(f"unexpected error: {e}")

   shiire_tanka
0         500.0
1         320.0
2        2800.0
3        2800.0
4        5000.0
5           NaN
6         790.0
7           NaN
   shiire_tanka
0           NaN
1         320.0
2         500.0
3        2800.0
4        5000.0
5         790.0


In [48]:
# distinction multiple columns
# distinction with null data
with psycopg2.connect(dst) as con, con.cursor() as cur:
    table_name = 'shohin'
    try:
        sql = f"""select shohin_bunrui, torokubi
                    from {table_name}"""
        print(pd.read_sql(sql, con=con))
        
        sql = f"""select distinct shohin_bunrui, torokubi
                    from {table_name}"""
        print(pd.read_sql(sql, con=con))
    except psycopg2.Error as e:
        print(f"unexpected error: {e}")

  shohin_bunrui    torokubi
0            衣服  2009-09-20
1          事務用品  2009-09-11
2            衣服        None
3        キッチン用品  2009-09-20
4        キッチン用品  2009-01-15
5        キッチン用品  2009-09-20
6        キッチン用品  2008-04-28
7          事務用品  2009-11-11
  shohin_bunrui    torokubi
0            衣服        None
1        キッチン用品  2009-01-15
2            衣服  2009-09-20
3        キッチン用品  2008-04-28
4          事務用品  2009-11-11
5          事務用品  2009-09-11
6        キッチン用品  2009-09-20


In [49]:
# where word for filtering rows
with psycopg2.connect(dst) as con, con.cursor() as cur:
    table_name = 'shohin'
    try:
        sql = f"""select *
                    from {table_name}
                    where shohin_bunrui = '衣服';"""
        print(pd.read_sql(sql, con=con))
    except psycopg2.Error as e:
        print(f"unexpected error: {e}")

  shohin_id shohin_mei shohin_bunrui  hanbai_tanka  shiire_tanka    torokubi
0      0001       Tシャツ            衣服          1000           500  2009-09-20
1      0003    カッターシャツ            衣服          4000          2800        None


In [54]:
# write comments
with psycopg2.connect(dst) as con, con.cursor() as cur:
    table_name = 'shohin'
    try:
        sql = f"""
                -- single line comment
                /* muliple
                    line
                    comment
                */
                select *
                    -- it's able to insert comments between SQL statements
                    from {table_name}
                    /* hey
                        hey
                            hey
                        */
                    where shohin_bunrui = '衣服';"""
        print(pd.read_sql(sql, con=con))
    except psycopg2.Error as e:
        print(f"unexpected error: {e}")

  shohin_id shohin_mei shohin_bunrui  hanbai_tanka  shiire_tanka    torokubi
0      0001       Tシャツ            衣服          1000           500  2009-09-20
1      0003    カッターシャツ            衣服          4000          2800        None
